In [1]:
import pandas as pd
import numpy as np

In [2]:
stats = pd.read_pickle('C:/Users/YAVUZ/Transfer Fee Project/National Stats Scraper/Final_National_Stats_Matches.pkl')

In [3]:
stats.shape

(256354, 19)

In [4]:
stats_unique=stats.drop_duplicates()

In [5]:
stats_unique.shape

(242872, 19)

In [6]:
national_urls = pd.read_pickle('C:/Users/YAVUZ/Transfer Fee Project/Feature Engineering/National_Stats_Features_All.pkl')

In [7]:
profile=pd.read_pickle('C:/Users/YAVUZ/Transfer Fee Project/Player_Profiler/Player_profile.pkl')

In [8]:
#pd.DataFrame(stats['position'].unique()).to_csv('position_group_national.csv')

In [9]:
pos_groups = pd.read_csv('./position_group_national.csv', delimiter=";")
pos_groups.columns = ['position', 'Excuse']

In [10]:
pos_groups

,position,Excuse
0,RW,Position
1,Tendonitis,Injury
2,not in squad,not in squad
3,LW,Position
4,NaN,XX
...,...,...
273,Bruise on ankle,Injury
274,Tibia and Fibula Fracture,Injury
275,Contracture,Injury
276,Fracture-dislocation of the ankle,Injury


In [11]:
stats_unique = stats_unique.merge(pos_groups, on=['position'], how='left')

In [12]:
stats_unique_joined = stats_unique.merge(national_urls[['Transfer_id','verein_id','Team']], on=['Transfer_id','verein_id'], how='left')

In [13]:
stats_unique_joined.shape

(256319, 21)

In [14]:
stats_df=stats_unique_joined[(stats_unique_joined['home_team']==stats_unique_joined['Team'])| (stats_unique_joined['away_team']==stats_unique_joined['Team']) |
                    ((stats_unique_joined['competition'].isin(['U20 Elite League','U17-Europameisterschaft 2013'])) & (stats_unique_joined['home_team'].str.contains("\(")) &
                    (stats_unique_joined['away_team'].str.contains("\(")) & ((stats_unique_joined.apply(lambda x: x.Team in x.away_team, axis=1)) | (stats_unique_joined.apply(lambda x: x.Team in x.home_team, axis=1)) )) |
                    (stats_unique_joined['verein_id']=='5148')] 

In [15]:
aa = stats_df['result'].str.extractall('(\d+)')
aa = aa.unstack()
aa.columns = ['Home_Goals','Away_Goals']
stats_df = stats_df.merge(aa,how='inner', left_index=True, right_index=True)

In [16]:
stats_df

,competition,match_date,home_team,away_team,result,position,goals,assists,own_goals,yellow_card,...,substitutions_off,minutes_played,Transfer_id,id,verein_id,Transfer_Date,Excuse,Team,Home_Goals,Away_Goals
0,International Friendlies,2013-08-13,Germany U21,France U21,0:0,RW,,,,,...,,30',1391587,7161,3817,2016-01-01,Position,Germany U21,0,0
1,UEFA European Under-21 Championship Qualifying,2013-09-06,Faroe U21,Germany U21,0:3,RW,1,1,,,...,,45',1391587,7161,3817,2016-01-01,Position,Germany U21,0,3
2,UEFA European Under-21 Championship Qualifying,2013-09-09,Ireland U21,Germany U21,0:4,RW,,,,,...,46',45',1391587,7161,3817,2016-01-01,Position,Germany U21,0,4
3,UEFA European Under-21 Championship Qualifying,2013-10-11,Germany U21,Montenegro U21,2:0,Tendonitis,NaN,NaN,NaN,NaN,...,NaN,NaN,1391587,7161,3817,2016-01-01,Injury,Germany U21,2,0
4,UEFA European Under-21 Championship Qualifying,2013-10-15,Germany U21,Faroe U21,3:2,not in squad,NaN,NaN,NaN,NaN,...,NaN,NaN,1391587,7161,3817,2016-01-01,not in squad,Germany U21,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256314,UEFA Nations League B,2020-09-04,Scotland,Israel,1:1,LB,,,,,...,,90',3104424,60761,5547,2020-10-16,Position,Israel,1,1
256315,UEFA Nations League B,2020-09-07,Israel,Slovakia,1:1,LB,,,,,...,71',71',3104424,60761,5547,2020-10-16,Position,Israel,1,1
256316,European Qualifiers Play-offs,2020-10-08,Scotland,Israel,5:3 on pens,Corona virus,NaN,NaN,NaN,NaN,...,NaN,NaN,3104424,60761,5547,2020-10-16,Injury,Israel,5,3
256317,UEFA Nations League B,2020-10-11,Israel,Czech Republic,1:2,Corona virus,NaN,NaN,NaN,NaN,...,NaN,NaN,3104424,60761,5547,2020-10-16,Injury,Israel,1,2


In [17]:
conditions = [(stats_df['home_team'] == stats_df['Team'])&(stats_df['Home_Goals']>stats_df['Away_Goals']),
              ((stats_df['competition'].isin(['U20 Elite League','U17-Europameisterschaft 2013'])) & ((stats_df.apply(lambda x: x.Team in x.home_team, axis=1))))&(stats_df['Home_Goals']>stats_df['Away_Goals']),
             (stats_df['home_team'] == 'Macedonia')&(stats_df['verein_id'] =='5148')&(stats_df['Home_Goals']>stats_df['Away_Goals']),
              
              
              (stats_df['home_team'] == stats_df['Team'])&(stats_df['Home_Goals']==stats_df['Away_Goals']),
              ((stats_df['competition'].isin(['U20 Elite League','U17-Europameisterschaft 2013'])) & ((stats_df.apply(lambda x: x.Team in x.home_team, axis=1))))&(stats_df['Home_Goals']==stats_df['Away_Goals']),
             (stats_df['home_team'] == 'Macedonia')&(stats_df['verein_id'] =='5148')&(stats_df['Home_Goals']==stats_df['Away_Goals']),

              
              
              
             (stats_df['home_team'] == stats_df['Team'])&(stats_df['Home_Goals']<stats_df['Away_Goals']),
             ((stats_df['competition'].isin(['U20 Elite League','U17-Europameisterschaft 2013'])) & ((stats_df.apply(lambda x: x.Team in x.home_team, axis=1))))&(stats_df['Home_Goals']<stats_df['Away_Goals']),
             (stats_df['home_team'] == 'Macedonia')&(stats_df['verein_id'] =='5148')&(stats_df['Home_Goals']<stats_df['Away_Goals']),
 
              
              
              
             (stats_df['away_team'] == stats_df['Team'])&(stats_df['Away_Goals']>stats_df['Home_Goals']),
              ((stats_df['competition'].isin(['U20 Elite League','U17-Europameisterschaft 2013'])) & ((stats_df.apply(lambda x: x.Team in x.away_team, axis=1))))&(stats_df['Away_Goals']>stats_df['Home_Goals']),
             (stats_df['away_team'] == 'Macedonia')&(stats_df['verein_id'] =='5148')&(stats_df['Away_Goals']>stats_df['Home_Goals']),
                            
              
              
              
              
             (stats_df['away_team'] == stats_df['Team'])&(stats_df['Away_Goals']==stats_df['Home_Goals']),
              ((stats_df['competition'].isin(['U20 Elite League','U17-Europameisterschaft 2013'])) & ((stats_df.apply(lambda x: x.Team in x.away_team, axis=1))))&(stats_df['Away_Goals']==stats_df['Home_Goals']),
             (stats_df['away_team'] == 'Macedonia')&(stats_df['verein_id'] =='5148')&(stats_df['Away_Goals']==stats_df['Home_Goals']),
                         
              
              
              
             (stats_df['away_team'] == stats_df['Team'])&(stats_df['Away_Goals']<stats_df['Home_Goals']),            
              ((stats_df['competition'].isin(['U20 Elite League','U17-Europameisterschaft 2013'])) & ((stats_df.apply(lambda x: x.Team in x.away_team, axis=1))))&(stats_df['Away_Goals']<stats_df['Home_Goals']),
             (stats_df['away_team'] == 'Macedonia')&(stats_df['verein_id'] =='5148')&(stats_df['Away_Goals']<stats_df['Home_Goals'])]
outputs = ['win','win','win','draw','draw','draw','lose','lose','lose','win','win','win','draw','draw','draw','lose','lose','lose']
stats_df['Win_Draw_Lose'] = np.select(conditions, outputs)

In [18]:
#transfer id içerisinde gelen maç sayısı
stats_df.sort_values(by=['Transfer_id','match_date'],ascending=False, inplace=True)
stats_df['match_count'] = stats_df.groupby('Transfer_id').cumcount()

In [19]:
stats_df['minutes_played'] = pd.to_numeric(stats_df['minutes_played'].str.replace('\'',''),  errors='coerce')
stats_df['substitutions_on'] = pd.to_numeric(stats_df['substitutions_on'].str.replace('\'',''),  errors='coerce')
stats_df['substitutions_off'] = pd.to_numeric(stats_df['substitutions_off'].str.replace('\'',''),  errors='coerce')
stats_df['goals'] = pd.to_numeric(stats_df['goals'], errors='coerce')
stats_df['assists'] = pd.to_numeric(stats_df['assists'], errors='coerce')
stats_df['own_goals'] = pd.to_numeric(stats_df['own_goals'], errors='coerce')
stats_df['yellow_card'] = pd.to_numeric(stats_df['yellow_card'].str.replace('\'',''), errors='coerce')
stats_df['second_yellow_card'] = pd.to_numeric(stats_df['second_yellow_card'].str.replace('\'',''), errors='coerce')
stats_df['red_card'] = pd.to_numeric(stats_df['red_card'].str.replace('\'',''), errors='coerce')
stats_df['Home_Goals'] = pd.to_numeric(stats_df['Home_Goals'], errors='coerce')
stats_df['Away_Goals'] = pd.to_numeric(stats_df['Away_Goals'], errors='coerce')

In [20]:
conditions = [stats_df['yellow_card']>=0]
output = [1]
stats_df['yellow_card_flag'] = np.select(conditions, output, default=0)

In [21]:
conditions = [stats_df['second_yellow_card']>=0]
output = [1]
stats_df['second_yellow_card_flag'] = np.select(conditions, output, default=0)

In [22]:
conditions = [stats_df['red_card']>=0]
output = [1]
stats_df['red_card_flag'] = np.select(conditions, output, default=0)

In [23]:
conditions = [stats_df['minutes_played']>=0]
output = [1]
stats_df['matches_played_flag'] = np.select(conditions, output, default=0)

In [24]:
conditions = [(stats_df['substitutions_on'].isna())&(stats_df['minutes_played']>0)]
output = [1]
stats_df['starting_lineup'] = np.select(conditions, output, default=0)


In [25]:
conditions = [stats_df['substitutions_off']>0]
output = [1]
stats_df['substitution_off_flag'] = np.select(conditions, output, default=0)

In [26]:
len(stats_df['Transfer_id'].unique())

6534

In [27]:
stats_df

,competition,match_date,home_team,away_team,result,position,goals,assists,own_goals,yellow_card,...,Home_Goals,Away_Goals,Win_Draw_Lose,match_count,yellow_card_flag,second_yellow_card_flag,red_card_flag,matches_played_flag,starting_lineup,substitution_off_flag
256192,UEFA Nations League B,2020-10-14,Finland,Ireland,1:0,on the bench,NaN,NaN,NaN,NaN,...,1,0,lose,0,0,0,0,0,0,0
256211,UEFA European Under-21 Championship Qualifying,2018-09-11,Ireland U21,Germany U21,0:6,RW,NaN,NaN,NaN,NaN,...,0,6,lose,1,0,0,0,1,1,1
256209,UEFA European Under-21 Championship Qualifying,2018-09-07,Kosovo U21,Ireland U21,1:1,LW,NaN,NaN,NaN,NaN,...,1,1,draw,2,0,0,0,1,1,1
256207,UEFA European Under-21 Championship Qualifying,2018-03-27,Ireland U21,Azerbaijan U21,1:0,AM,NaN,NaN,NaN,74.0,...,1,0,win,3,1,0,0,1,1,0
256205,International Friendlies,2018-03-22,Ireland U21,Iceland U21,3:1,AM,1.0,NaN,NaN,NaN,...,3,1,win,4,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14765,International Friendlies,2014-10-14,Brazil,Japan,4:0,on the bench,NaN,NaN,NaN,NaN,...,4,0,win,0,0,0,0,0,0,0
14764,International Friendlies,2014-10-11,Brazil,Argentina,2:0,on the bench,NaN,NaN,NaN,NaN,...,2,0,win,1,0,0,0,0,0,0
14763,U17-Weltmeisterschaft 2009,2009-10-30,Switzerland U17,Brazil U17,1:0,on the bench,NaN,NaN,NaN,NaN,...,1,0,lose,2,0,0,0,0,0,0
14762,U17-Weltmeisterschaft 2009,2009-10-27,Brazil U17,Mexico U17,0:1,on the bench,NaN,NaN,NaN,NaN,...,0,1,lose,3,0,0,0,0,0,0


In [28]:
def stats_features(count):
    unique_transfers = stats_df['Transfer_id'].unique()
    base_df = pd.DataFrame(unique_transfers, columns =['Transfer_id']).set_index('Transfer_id')
    lastx = stats_df[stats_df['match_count']<count]
    match_count = lastx[['Transfer_id','match_date']].groupby('Transfer_id').count()
    base_df = base_df.merge(match_count, on='Transfer_id', how = 'left')
    rate_start_lineup = lastx[['Transfer_id','starting_lineup']].groupby('Transfer_id').sum()
    base_df = base_df.merge(rate_start_lineup, left_index=True, right_index=True, how='left')
    games_played = lastx[['Transfer_id','matches_played_flag']].groupby('Transfer_id').sum()
    base_df = base_df.merge(games_played, left_index=True, right_index=True, how='left')
    cols = ['Transfer_id','goals', 'assists', 'own_goals', 'yellow_card_flag',
       'second_yellow_card_flag', 'red_card_flag', 'minutes_played','substitution_off_flag']
    lastx_stats =lastx[cols].groupby('Transfer_id').aggregate({'goals':'sum', 'assists':'sum', 'own_goals':'sum',\
            'yellow_card_flag':'sum', 'second_yellow_card_flag':'sum','red_card_flag':'sum',\
            'minutes_played':'sum','substitution_off_flag':'sum'})
    base_df = base_df.merge(lastx_stats, left_index=True, right_index=True, how='left')
    base_df['match_played_rate'] = round(base_df['matches_played_flag'] / base_df['match_date'], 2)

    
    
    
    
    
    base_df['rate_subs_off'] = round(base_df['substitution_off_flag'] / base_df['starting_lineup'], 2)
    conditions = [lastx['Win_Draw_Lose'] =='win',lastx['Win_Draw_Lose'] =='draw']
    outputs = [3,1]
    lastx['match_points'] = np.select(conditions, outputs, default=0)
    total_points_played = lastx.loc[lastx['matches_played_flag']==1,['Transfer_id','match_points']].groupby('Transfer_id').sum().add_suffix('_played')
    total_points_notplayed = lastx.loc[lastx['matches_played_flag']==0,['Transfer_id','match_points']].groupby('Transfer_id').sum().add_suffix('_notplayed')
    base_df = base_df.merge(total_points_played, left_index = True, right_index=True, how='left')
    base_df = base_df.merge(total_points_notplayed, left_index = True, right_index=True, how='left')
    team_away_goals = lastx.loc[lastx['Team'] == lastx['away_team'],['Transfer_id','Away_Goals']].groupby('Transfer_id').sum().add_suffix('_team')
    team_home_goals = lastx.loc[lastx['Team'] == lastx['home_team'],['Transfer_id','Home_Goals']].groupby('Transfer_id').sum().add_suffix('_team')
    team_away_goals_played = lastx.loc[(lastx['Team'] == lastx['away_team'])&(lastx['matches_played_flag']==1),['Transfer_id','Away_Goals']].groupby('Transfer_id').sum().add_suffix('_team_played')
    team_home_goals_played = lastx.loc[(lastx['Team'] == lastx['home_team'])&(lastx['matches_played_flag']==1),['Transfer_id','Home_Goals']].groupby('Transfer_id').sum().add_suffix('_team_played')  
    base_df = base_df.merge(team_home_goals, left_index=True, right_index=True, how='left')
    base_df = base_df.merge(team_away_goals, left_index=True, right_index=True, how='left')
    base_df = base_df.merge(team_home_goals_played, left_index=True, right_index=True, how='left')
    base_df = base_df.merge(team_away_goals_played, left_index=True, right_index=True, how='left')
    base_df['total_goals_team'] = base_df['Home_Goals_team'] + base_df['Away_Goals_team']
    base_df['total_goals_team_played'] = base_df['Home_Goals_team_played'] + base_df['Away_Goals_team_played']
    team_away_conceded_goals = lastx.loc[lastx['Team'] == lastx['home_team'],['Transfer_id','Away_Goals']].groupby('Transfer_id').sum().add_suffix('_team_conceded')
    team_home_conceded_goals = lastx.loc[lastx['Team'] == lastx['away_team'],['Transfer_id','Home_Goals']].groupby('Transfer_id').sum().add_suffix('_team_conceded')
    team_away_conceded_goals_played = lastx.loc[(lastx['Team'] == lastx['home_team'])&(lastx['matches_played_flag']==1),['Transfer_id','Away_Goals']].groupby('Transfer_id').sum().add_suffix('_team_conceded_played')
    team_home_conceded_goals_played = lastx.loc[(lastx['Team'] == lastx['away_team'])&(lastx['matches_played_flag']==1),['Transfer_id','Home_Goals']].groupby('Transfer_id').sum().add_suffix('_team_conceded_played')
    base_df = base_df.merge(team_away_conceded_goals, left_index=True, right_index=True, how='left')
    base_df = base_df.merge(team_home_conceded_goals, left_index=True, right_index=True, how='left')
    base_df = base_df.merge(team_away_conceded_goals_played, left_index=True, right_index=True, how='left')
    base_df = base_df.merge(team_home_conceded_goals_played, left_index=True, right_index=True, how='left')
    base_df['total_goals_team_conceded'] = base_df['Home_Goals_team_conceded'] + base_df['Away_Goals_team_conceded']
    base_df['total_goals_team_conceded_played'] = base_df['Home_Goals_team_conceded_played'] + base_df['Away_Goals_team_conceded_played']
    

    conditions = [((lastx['Team'] == lastx['home_team']) & (lastx['Away_Goals']==0)) , ((lastx['Team'] == lastx['away_team']) & (lastx['Home_Goals']==0)) ]
    outputs = [1,1]
    lastx['clean_sheet_flg'] = np.select(conditions, outputs, default=0)    
    total_clean_sheets_played=lastx.loc[lastx['matches_played_flag']==1,['Transfer_id','clean_sheet_flg']].groupby('Transfer_id').sum().add_suffix('_played')
    total_clean_sheets_not_played=lastx.loc[lastx['matches_played_flag']==0,['Transfer_id','clean_sheet_flg']].groupby('Transfer_id').sum().add_suffix('_notplayed')
    base_df = base_df.merge(total_clean_sheets_played, left_index=True, right_index=True, how='left')
    base_df = base_df.merge(total_clean_sheets_not_played, left_index=True, right_index=True, how='left')
    
    
    #Played per game Features
    base_df['lineup_rate'] = round(base_df['starting_lineup'] / base_df['matches_played_flag'], 2)
    base_df['minutes_per_game'] = round(base_df['minutes_played'] / base_df['matches_played_flag'],2)
    
    base_df['goals_per_Game'] = round(base_df['goals'] / base_df['matches_played_flag'],2)
    
    base_df['assists_per_game'] = round(base_df['assists'] / base_df['matches_played_flag'],2)
    
    base_df['own_goals_per_game'] = round(base_df['own_goals'] / base_df['matches_played_flag'],2)
    
    base_df['yellow_cards_per_game'] = round(base_df['yellow_card_flag'] / base_df['matches_played_flag'],2)
    
    base_df['second_yellow_card_per_game'] = round(base_df['second_yellow_card_flag'] / base_df['matches_played_flag'],2)
    
    base_df['red_card_per_game'] = round(base_df['red_card_flag'] / base_df['matches_played_flag'],2)
    
    base_df['team_goals_per_game'] = round(base_df['total_goals_team_played'] / base_df['matches_played_flag'],2)
    
    base_df['points_per_game'] = round(base_df['match_points_played'] / base_df['matches_played_flag'],2)
    
    base_df['team_conceeded_goals_per_game'] = round(base_df['total_goals_team_conceded_played'] / base_df['matches_played_flag'],2)
    
    base_df['clean_sheet_per_game'] = round(base_df['clean_sheet_flg_played'] / base_df['matches_played_flag'],2)
    
    
    #Not Played per game Features
    base_df['team_goals_per_game_not_played'] = round((base_df['total_goals_team'] - base_df['total_goals_team_played'])/(base_df['match_date']-base_df['matches_played_flag']),2)
    
    base_df['clean_sheet_per_game_not_played'] = round(base_df['clean_sheet_flg_notplayed'] / (base_df['match_date']-base_df['matches_played_flag']),2)
    
    base_df['team_conceeded_goals_per_game_not_played'] = round((base_df['total_goals_team_conceded'] - base_df['total_goals_team_conceded_played'])/(base_df['match_date']-base_df['matches_played_flag']),2)
    base_df['points_per_game_not_played'] = round(base_df['match_points_notplayed']/(base_df['match_date']-base_df['matches_played_flag']),2)
    
    #Coverage Features
    base_df['minutes_played_coverage_played'] =round(base_df['minutes_played']/(90*base_df['matches_played_flag']),2)
    base_df['minutes_played_coverage_all'] =round(base_df['minutes_played']/(90*base_df['match_date']),2)    
    base_df['points_played_coverage'] = round(base_df['match_points_played'] / (3*base_df['matches_played_flag']),2)   
    base_df['points_not_played_coverage'] = round(base_df['match_points_notplayed'] / (3*(base_df['match_date']-base_df['matches_played_flag'])),2)
    
    
    #Injury/Suspenion/bench/not in squad rate
    pos_count = lastx[['Transfer_id','Excuse','match_date']].groupby(['Transfer_id','Excuse']).count().reset_index()
    pos_count_transpose = pd.pivot_table(pos_count, values = 'match_date', index=['Transfer_id'], columns = 'Excuse').reset_index()
    pos_count_final = pos_count_transpose[['Transfer_id','Injury','Suspension','not in squad','on the bench']]
    pos_count_final = pos_count_final.reset_index(drop=True).set_index('Transfer_id')
    base_df = base_df.merge(pos_count_final, left_index=True, right_index=True,how='left')
    
    base_df['injury_rate'] = round(base_df['Injury']/base_df['match_date'],2)
    
    base_df['suspension_rate'] = round(base_df['Suspension']/base_df['match_date'],2)
    
    base_df['on_the_bench_rate'] = round(base_df['on the bench']/base_df['match_date'],2)
    
    base_df['not_in_squad_rate'] = round(base_df['not in squad']/base_df['match_date'],2)
    
    #Min_date/recency
    transfer_date=lastx[['Transfer_id','Transfer_Date']].drop_duplicates().set_index('Transfer_id')
    base_df = base_df.merge(transfer_date, left_index=True, right_index=True,how='left')
    min_date=lastx[['Transfer_id','match_date']].groupby('Transfer_id').min().add_suffix('_min')
    base_df = base_df.merge(min_date, left_index=True, right_index=True,how='left')
    base_df['last_played_game_recency']=base_df['Transfer_Date']-base_df['match_date_min']
    
    
    #drop unnecessary columns
    
    base_df.drop(['Transfer_Date', 'match_date_min','Away_Goals_team_conceded','Home_Goals_team_conceded', 'Away_Goals_team_conceded_played','Home_Goals_team_conceded_played','Home_Goals_team', 'Away_Goals_team','Home_Goals_team_played', 'Away_Goals_team_played'], axis=1, inplace=True)
    base_df=base_df.add_suffix('_National_'+str(count))
    return base_df

In [29]:
stats_features_30=stats_features(30)

<ipython-input-28-ca912d4c6450>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lastx['match_points'] = np.select(conditions, outputs, default=0)
<ipython-input-28-ca912d4c6450>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lastx['clean_sheet_flg'] = np.select(conditions, outputs, default=0)


In [30]:
stats_features_20=stats_features(20)

<ipython-input-28-ca912d4c6450>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lastx['match_points'] = np.select(conditions, outputs, default=0)
<ipython-input-28-ca912d4c6450>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lastx['clean_sheet_flg'] = np.select(conditions, outputs, default=0)


In [31]:
stats_features_10=stats_features(10)

<ipython-input-28-ca912d4c6450>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lastx['match_points'] = np.select(conditions, outputs, default=0)
<ipython-input-28-ca912d4c6450>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lastx['clean_sheet_flg'] = np.select(conditions, outputs, default=0)


In [32]:
stats_features_5=stats_features(5)

<ipython-input-28-ca912d4c6450>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lastx['match_points'] = np.select(conditions, outputs, default=0)
<ipython-input-28-ca912d4c6450>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lastx['clean_sheet_flg'] = np.select(conditions, outputs, default=0)


In [33]:
from functools import reduce
data_frames = [stats_features_30,stats_features_20, stats_features_10, stats_features_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Transfer_id'],
                                            how='outer'), data_frames)

In [34]:
df_merged.to_pickle('./National_Stats_Features_All.pkl')